In [233]:
import json
import numpy as np
import pandas as pd
from inline_sql import sql
import pandasql as ps
import matplotlib.pyplot as plt

### Importacion de tablas

In [234]:
pozos = pd.read_csv("pozos-con-geo-info.csv")
produccion = pd.read_csv("produccin-de-pozos-de-gas-y-petrleo-2024.csv")

/var/folders/r_/x2vzgchn7xg93cptq09838d91bmbgx/T/ipykernel_5080/1715450679.py:2: DtypeWarning: Columns (16,36) have mixed types. Specify dtype option on import or set low_memory=False.
  produccion = pd.read_csv("produccin-de-pozos-de-gas-y-petrleo-2024.csv")


In [235]:
pozos.info(), produccion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84332 entries, 0 to 84331
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               84332 non-null  int64  
 1   sigla                    84332 non-null  object 
 2   idpozo                   84332 non-null  int64  
 3   area                     84332 non-null  object 
 4   cod_area                 84332 non-null  object 
 5   empresa                  83435 non-null  object 
 6   yacimiento               84332 non-null  object 
 7   cod_yacimiento           84332 non-null  object 
 8   formacion                81517 non-null  object 
 9   cuenca                   84332 non-null  object 
 10  provincia                84332 non-null  object 
 11  cota                     84332 non-null  float64
 12  profundidad              84332 non-null  float64
 13  clasificacion            84332 non-null  object 
 14  subclasificacion      

(None, None)

### Rellado con Estado Nacional en provincia blanks

In [236]:
pozos['provincia_nombre'] = pozos['provincia_nombre'].fillna('Estado Nacional')
pozos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84332 entries, 0 to 84331
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               84332 non-null  int64  
 1   sigla                    84332 non-null  object 
 2   idpozo                   84332 non-null  int64  
 3   area                     84332 non-null  object 
 4   cod_area                 84332 non-null  object 
 5   empresa                  83435 non-null  object 
 6   yacimiento               84332 non-null  object 
 7   cod_yacimiento           84332 non-null  object 
 8   formacion                81517 non-null  object 
 9   cuenca                   84332 non-null  object 
 10  provincia                84332 non-null  object 
 11  cota                     84332 non-null  float64
 12  profundidad              84332 non-null  float64
 13  clasificacion            84332 non-null  object 
 14  subclasificacion      

### Limpieza pozos no productivos

Tomo los idpozos que tuvieron algo de producción en algún mes del año

In [237]:
produccion['index'] = produccion.index
# Calcula las sumas de prod_pet y prod_gas agrupadas por idpozo
sums = produccion.groupby('idpozo')[['prod_pet', 'prod_gas']].sum()

# Filtra idpozo donde al menos una columna tenga suma distinta de 0
non_null_pozo_ids = sums[(sums['prod_pet'] != 0) | (sums['prod_gas'] != 0)].index

# Filtra el DataFrame original
produccion_nonula = produccion[produccion['idpozo'].isin(non_null_pozo_ids)]

# Muestra información del resultado
produccion_nonula.info()

<class 'pandas.core.frame.DataFrame'>
Index: 274496 entries, 250 to 818738
Data columns (total 39 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   idempresa               274496 non-null  object 
 1   anio                    274496 non-null  int64  
 2   mes                     274496 non-null  int64  
 3   idpozo                  274496 non-null  int64  
 4   prod_pet                274496 non-null  float64
 5   prod_gas                274496 non-null  float64
 6   prod_agua               274496 non-null  float64
 7   iny_agua                274496 non-null  float64
 8   iny_gas                 274496 non-null  float64
 9   iny_co2                 274496 non-null  float64
 10  iny_otro                274496 non-null  float64
 11  tef                     274496 non-null  float64
 12  vida_util               90 non-null      float64
 13  tipoextraccion          274474 non-null  object 
 14  tipoestado             

Filtro en la tabla de pozos los que no tuvieron nada de producción

In [238]:
pozos = pozos[pozos['idpozo'].isin(produccion_nonula['idpozo'].unique())]
pozos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27727 entries, 0 to 84330
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               27727 non-null  int64  
 1   sigla                    27727 non-null  object 
 2   idpozo                   27727 non-null  int64  
 3   area                     27727 non-null  object 
 4   cod_area                 27727 non-null  object 
 5   empresa                  27727 non-null  object 
 6   yacimiento               27727 non-null  object 
 7   cod_yacimiento           27727 non-null  object 
 8   formacion                27699 non-null  object 
 9   cuenca                   27727 non-null  object 
 10  provincia                27727 non-null  object 
 11  cota                     27727 non-null  float64
 12  profundidad              27727 non-null  float64
 13  clasificacion            27727 non-null  object 
 14  subclasificacion         27

# Unicidad sigla-formacion

Veo duplicados en sigla y formacion. Un duplicado difiere en la provincia (Santa Cruz / Tierra del fuego), o difiere en la empresa. Las producciones son practicamente las mismas, muy parecidas, podemos decir que los duplicados son el mismo pozo.

In [240]:
duplicados = pozos[pozos[['sigla','formacion']].duplicated(keep=False)]
duplicados.to_csv("duplicados_sigla_form.csv")
duplicados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 3381 to 60478
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               80 non-null     int64  
 1   sigla                    80 non-null     object 
 2   idpozo                   80 non-null     int64  
 3   area                     80 non-null     object 
 4   cod_area                 80 non-null     object 
 5   empresa                  80 non-null     object 
 6   yacimiento               80 non-null     object 
 7   cod_yacimiento           80 non-null     object 
 8   formacion                80 non-null     object 
 9   cuenca                   80 non-null     object 
 10  provincia                80 non-null     object 
 11  cota                     80 non-null     float64
 12  profundidad              80 non-null     float64
 13  clasificacion            80 non-null     object 
 14  subclasificacion         80

In [241]:
def merge_provincias_api (pozos):
    indexes = []
    siglas = pozos['sigla'].unique()
    for sigla in siglas:
        pozos_sigla = pozos[pozos['sigla'] == sigla]
        provi_dist = len(pozos_sigla['provincia'].unique())
        if (provi_dist > 1):
            ids = pozos_sigla['idpozo']
            for i in (pozos[pozos['idpozo'].isin(ids)].index):
                pozos.loc[i, 'provincia'] = pozos_sigla['provincia_nombre'].iloc[0]
                indexes.append(i)
            indexes.pop()
    return indexes

In [242]:
indexes_borrados = merge_provincias_api(pozos)

In [243]:
print(len(pozos))
pozosaborrar = pozos[pozos.index.isin(indexes_borrados)]
pozos = pozos[~pozos.index.isin(indexes_borrados)]
len(indexes_borrados)
print(len(pozos))

pozos = pozos[~pozos.index.isin(indexes_borrados)]
print(len(produccion_nonula))

produccion_nonula = produccion_nonula[~produccion_nonula['idpozo'].isin(pozosaborrar['idpozo'])]
print(len(produccion_nonula))



27727
27695
274496
274176


In [244]:
produccion_nonula.to_csv("prod-no-nula.csv")
duplicados = pozos[pozos[['sigla','formacion']].duplicated(keep=False)]
duplicados.to_csv("duplicados_sigla_form_2.csv")
duplicados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 3381 to 51770
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               16 non-null     int64  
 1   sigla                    16 non-null     object 
 2   idpozo                   16 non-null     int64  
 3   area                     16 non-null     object 
 4   cod_area                 16 non-null     object 
 5   empresa                  16 non-null     object 
 6   yacimiento               16 non-null     object 
 7   cod_yacimiento           16 non-null     object 
 8   formacion                16 non-null     object 
 9   cuenca                   16 non-null     object 
 10  provincia                16 non-null     object 
 11  cota                     16 non-null     float64
 12  profundidad              16 non-null     float64
 13  clasificacion            16 non-null     object 
 14  subclasificacion         16

In [246]:
produccion_nonula[produccion_nonula['idpozo'].isin(duplicados['idpozo'])].to_csv("duplicados_sigla_form_prod.csv")

# Unicidad de sigla

Me fijo de los que tienen distintas coordenadas, cuales tienen misma boca. Esperaríamos que no haya ninguna por lo mismo de antes, si no hay coordenadas repetidas, en principio no debería haber dos siglas iguales. Sin embargo hay 294

In [213]:
sigla_dup = pozos[(pozos['sigla'].duplicated(keep=False))]
sigla_dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2486 entries, 224 to 84301
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               2486 non-null   int64  
 1   sigla                    2486 non-null   object 
 2   idpozo                   2486 non-null   int64  
 3   area                     2486 non-null   object 
 4   cod_area                 2486 non-null   object 
 5   empresa                  2486 non-null   object 
 6   yacimiento               2486 non-null   object 
 7   cod_yacimiento           2486 non-null   object 
 8   formacion                2486 non-null   object 
 9   cuenca                   2486 non-null   object 
 10  provincia                2486 non-null   object 
 11  cota                     2486 non-null   float64
 12  profundidad              2486 non-null   float64
 13  clasificacion            2486 non-null   object 
 14  subclasificacion         2

In [214]:
def coords_notclose (df):
    siglas = df['sigla'].unique()
    ids = []
    
    for sigla in siglas:
        pozos_eqsigla = df[df['sigla'] == sigla]
        lon_close = np.isclose(pozos_eqsigla['lon'], pozos_eqsigla['lon'].iloc[0], atol=1e-3)
        lat_close = np.isclose(pozos_eqsigla['lat'], pozos_eqsigla['lat'].iloc[0], atol=1e-3)
        
        for p in range (len(lon_close)):
            if not (lon_close[p] & lat_close[p]):
                for i in range (len(lon_close)):
                    ids.append(pozos_eqsigla['idpozo'].iloc[i])
                break
                
    return ids

dup_sigla_notclosecoord = sigla_dup[sigla_dup['idpozo'].isin(coords_notclose(sigla_dup))]
dup_sigla_notclosecoord.to_csv("duplicados_sigla_notclosecoord.csv")
dup_sigla_notclosecoord.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 4613 to 83009
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               25 non-null     int64  
 1   sigla                    25 non-null     object 
 2   idpozo                   25 non-null     int64  
 3   area                     25 non-null     object 
 4   cod_area                 25 non-null     object 
 5   empresa                  25 non-null     object 
 6   yacimiento               25 non-null     object 
 7   cod_yacimiento           25 non-null     object 
 8   formacion                25 non-null     object 
 9   cuenca                   25 non-null     object 
 10  provincia                25 non-null     object 
 11  cota                     25 non-null     float64
 12  profundidad              25 non-null     float64
 13  clasificacion            25 non-null     object 
 14  subclasificacion         25

In [215]:
import pandas as pd
import json
def make_coordinates_unique (pozos, duplicados):
    siglas = duplicados['sigla'].unique()
    for sigla in siglas:
        pozos_sigla = pozos[pozos['sigla'] == sigla]
        ids = pozos_sigla['idpozo']
        geojson = pozos_sigla['geojson'].iloc[0]
        geom = pozos_sigla['geom'].iloc[0]
        for i in (pozos[pozos['idpozo'].isin(ids)].index):                  
            pozos.loc[i, 'geojson'] = geojson
            pozos.loc[i, 'geom'] = geom
            
def recalculate_coordinates(df):
    # Eliminar columnas específicas si existen
    df.drop(columns=['coordinates', 'coordinates_str', 'lon', 'lat'], axis=1, errors='ignore', inplace=True)
    
    # Crear una columna temporal para procesar el contenido sin alterar 'geojson'
    df['geojson_dic'] = df['geojson'].apply(lambda x: json.loads(x.replace("'", '"')))
    
    # Extraer las coordenadas en nuevas columnas 'lon' y 'lat'
    df[['lon', 'lat']] = pd.DataFrame(
        df['geojson_dic'].apply(lambda x: x['coordinates']).to_list(), 
        index=df.index
    )
    
    # Eliminar la columna temporal para mantener limpio el DataFrame
    df.drop(columns=['geojson_dic'], errors='ignore', inplace=True)
   

A los que tienen distinta coordenada, los unifico (su geojson) y luego recalculo las coordenadas para tenerlas bien

In [216]:
make_coordinates_unique(pozos, dup_sigla_notclosecoord)
recalculate_coordinates(pozos)

# Unicidad de coordenadas

Me fijo geoms repetidos

In [217]:
geom_dup = pozos[pozos['geom'].duplicated(keep=False)]
geom_dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2376 entries, 239 to 84298
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               2376 non-null   int64  
 1   sigla                    2376 non-null   object 
 2   idpozo                   2376 non-null   int64  
 3   area                     2376 non-null   object 
 4   cod_area                 2376 non-null   object 
 5   empresa                  2376 non-null   object 
 6   yacimiento               2376 non-null   object 
 7   cod_yacimiento           2376 non-null   object 
 8   formacion                2376 non-null   object 
 9   cuenca                   2376 non-null   object 
 10  provincia                2376 non-null   object 
 11  cota                     2376 non-null   float64
 12  profundidad              2376 non-null   float64
 13  clasificacion            2376 non-null   object 
 14  subclasificacion         2

Me fijo geojsons repetidos

In [218]:
geojson_dup = pozos[pozos['geojson'].duplicated(keep=False)]
geojson_dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2376 entries, 239 to 84298
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               2376 non-null   int64  
 1   sigla                    2376 non-null   object 
 2   idpozo                   2376 non-null   int64  
 3   area                     2376 non-null   object 
 4   cod_area                 2376 non-null   object 
 5   empresa                  2376 non-null   object 
 6   yacimiento               2376 non-null   object 
 7   cod_yacimiento           2376 non-null   object 
 8   formacion                2376 non-null   object 
 9   cuenca                   2376 non-null   object 
 10  provincia                2376 non-null   object 
 11  cota                     2376 non-null   float64
 12  profundidad              2376 non-null   float64
 13  clasificacion            2376 non-null   object 
 14  subclasificacion         2

In [219]:
# Checkeo geom = geojson
False in (geom_dup.index == geojson_dup.index)

False

In [220]:
# Cantidad de coordenadas unicas
len(geom_dup['geom'].unique())

1101

### Ejemplo duplicado

In [221]:
geom_dup[(geom_dup['geom'] == geom_dup['geom'].iloc[500])]

,Unnamed: 0,sigla,idpozo,area,cod_area,empresa,yacimiento,cod_yacimiento,formacion,cuenca,...,type,info_coordenadas,departamento_id,departamento_nombre,municipio_id,municipio_nombre,provincia_id,provincia_nombre,lon,lat
42361,42361,YPF.Md.NDB-145,110116,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,DESFILADERO BAYO,DBA,rayoso,NEUQUINA,...,Point,"{'departamento_id': '50077', 'departamento_nom...",50077.0,Malargüe,500077.0,Malargüe,50.0,Mendoza,-69.017669,-37.328795
42362,42362,YPF.Md.NDB-145,110117,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,DESFILADERO BAYO,DBA,agrio,NEUQUINA,...,Point,"{'departamento_id': '50077', 'departamento_nom...",50077.0,Malargüe,500077.0,Malargüe,50.0,Mendoza,-69.017669,-37.328795


In [222]:
sample_dup = produccion[(produccion['idpozo'] == 109903) | (produccion['idpozo'] == 109904)]
sample_dup[(sample_dup['prod_pet'] != 0) & (sample_dup['prod_gas'] != 0)]

,idempresa,anio,mes,idpozo,prod_pet,prod_gas,prod_agua,iny_agua,iny_gas,iny_co2,...,areayacimiento,cuenca,provincia,tipo_de_recurso,proyecto,clasificacion,subclasificacion,sub_tipo_recurso,fecha_data,index
36661,YPF,2024,1,109904,15.78,0.07,161.95,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-01-31,36661
36662,YPF,2024,1,109903,36.83,0.15,377.89,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-01-31,36662
75000,YPF,2024,2,109904,13.38,0.06,157.97,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-02-29,75000
75001,YPF,2024,2,109903,31.22,0.15,368.58,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-02-29,75001
112625,YPF,2024,3,109904,15.13,0.07,172.38,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-03-31,112625
112626,YPF,2024,3,109903,35.30,0.16,402.23,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-03-31,112626
154639,YPF,2024,4,109904,15.16,0.07,174.34,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-04-30,154639
154640,YPF,2024,4,109903,35.38,0.16,406.80,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-04-30,154640
190633,YPF,2024,5,109904,12.93,0.06,153.19,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-05-31,190633
190634,YPF,2024,5,109903,30.18,0.14,357.43,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-05-31,190634


In [223]:
geom_dup.loc[42188] == geom_dup.loc[42189]

Unnamed: 0                 False
sigla                       True
idpozo                     False
area                        True
cod_area                    True
empresa                     True
yacimiento                  True
cod_yacimiento              True
formacion                  False
cuenca                      True
provincia                   True
cota                        True
profundidad                 True
clasificacion               True
subclasificacion            True
tipo_recurso                True
sub_tipo_recurso            True
gasplus                     True
tipopozo                    True
tipoextraccion              True
tipoestado                  True
adjiv_fecha_inicio_perf     True
adjiv_fecha_fin_perf        True
adjiv_fecha_inicio_term     True
adjiv_fecha_fin_term        True
geojson                     True
geom                        True
type                        True
info_coordenadas            True
departamento_id             True
departamen

### Checkeo cuáles tienen bocas unicas

Me fijo cuáles tienen bocas unicas dentro de mismas coordenadas. Esperaríamos que no haya ninguna ya que una boca es un punto fisico en un terreno, por lo que no debería existir una boca unica en una lista de coordenadas repetidas. Sin embargo hay 155

In [224]:
geom_dup_uneqbocas = geom_dup[~geom_dup['sigla'].duplicated(keep=False)]
geom_dup_uneqbocas.to_csv("duplicados_eqcoord_uneqsigla.csv")
geom_dup_uneqbocas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187 entries, 655 to 83928
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               187 non-null    int64  
 1   sigla                    187 non-null    object 
 2   idpozo                   187 non-null    int64  
 3   area                     187 non-null    object 
 4   cod_area                 187 non-null    object 
 5   empresa                  187 non-null    object 
 6   yacimiento               187 non-null    object 
 7   cod_yacimiento           187 non-null    object 
 8   formacion                187 non-null    object 
 9   cuenca                   187 non-null    object 
 10  provincia                187 non-null    object 
 11  cota                     187 non-null    float64
 12  profundidad              187 non-null    float64
 13  clasificacion            187 non-null    object 
 14  subclasificacion         18

Un ejemplo, difiere solo en formacion y sigla, es posible que hayan anotado diferente al anotar distintas formaciones

In [225]:
geom_dup[geom_dup['geom'] == geom_dup_uneqbocas['geom'].iloc[150]]

,Unnamed: 0,sigla,idpozo,area,cod_area,empresa,yacimiento,cod_yacimiento,formacion,cuenca,...,type,info_coordenadas,departamento_id,departamento_nombre,municipio_id,municipio_nombre,provincia_id,provincia_nombre,lon,lat
12849,12849,PNR.NQ.BE-7,50021,AL SUR DE LA DORSAL,SDD,OILSTONE ENERGIA S.A.,BORDE ESPINOSO,BEP,lotena,NEUQUINA,...,Point,"{'departamento_id': '58035', 'departamento_nom...",58035.0,Confluencia,NaN,NaN,58.0,Neuquén,-69.065563,-39.115653
76571,76571,Pi.Nq.BE-7,157679,AL SUR DE LA DORSAL,SDD,OILSTONE ENERGIA S.A.,BORDE ESPINOSO,BEP,vaca muerta,NEUQUINA,...,Point,"{'departamento_id': '58035', 'departamento_nom...",58035.0,Confluencia,NaN,NaN,58.0,Neuquén,-69.065563,-39.115653


In [226]:
pozos.to_csv("pozos-limpios.csv")

In [227]:
pozos

,Unnamed: 0,sigla,idpozo,area,cod_area,empresa,yacimiento,cod_yacimiento,formacion,cuenca,...,type,info_coordenadas,departamento_id,departamento_nombre,municipio_id,municipio_nombre,provincia_id,provincia_nombre,lon,lat
0,0,CH.CH.EaLE.x-1,212,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,...,Point,"{'departamento_id': '26021', 'departamento_nom...",26021.0,Escalante,NaN,NaN,26.0,Chubut,-68.287853,-45.591133
2,2,CH.CH.EaLE-3,214,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,...,Point,"{'departamento_id': '26021', 'departamento_nom...",26021.0,Escalante,NaN,NaN,26.0,Chubut,-68.283878,-45.590244
3,3,CH.CH.EaLE-4,215,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,...,Point,"{'departamento_id': '26021', 'departamento_nom...",26021.0,Escalante,NaN,NaN,26.0,Chubut,-68.289483,-45.594069
4,4,CH.CH.EaLE-5,216,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,...,Point,"{'departamento_id': '26021', 'departamento_nom...",26021.0,Escalante,NaN,NaN,26.0,Chubut,-68.294475,-45.594450
5,5,CH.CH.EaLE-6,217,ESTANCIA LA ESCONDIDA,ECH,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ELA,comodoro rivadavia,GOLFO SAN JORGE,...,Point,"{'departamento_id': '26021', 'departamento_nom...",26021.0,Escalante,NaN,NaN,26.0,Chubut,-68.297317,-45.592686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84326,84326,SHE.Nq.SB-1045(h),165739,SIERRAS BLANCAS,SBLA,SHELL ARGENTINA S.A.,SIERRAS BLANCAS,SIBL,vaca muerta,NEUQUINA,...,Point,"{'departamento_id': '58014', 'departamento_nom...",58014.0,Añelo,NaN,NaN,58.0,Neuquén,-68.437985,-38.471766
84327,84327,SHE.Nq.SB-1046(h),165740,SIERRAS BLANCAS,SBLA,SHELL ARGENTINA S.A.,SIERRAS BLANCAS,SIBL,vaca muerta,NEUQUINA,...,Point,"{'departamento_id': '58014', 'departamento_nom...",58014.0,Añelo,NaN,NaN,58.0,Neuquén,-68.437813,-38.471765
84328,84328,SHE.Nq.SB-1047(h),165741,SIERRAS BLANCAS,SBLA,SHELL ARGENTINA S.A.,SIERRAS BLANCAS,SIBL,vaca muerta,NEUQUINA,...,Point,"{'departamento_id': '58014', 'departamento_nom...",58014.0,Añelo,NaN,NaN,58.0,Neuquén,-68.437642,-38.471764
84329,84329,SHE.Nq.SB-1048(h),165742,SIERRAS BLANCAS,SBLA,SHELL ARGENTINA S.A.,SIERRAS BLANCAS,SIBL,vaca muerta,NEUQUINA,...,Point,"{'departamento_id': '58014', 'departamento_nom...",58014.0,Añelo,NaN,NaN,58.0,Neuquén,-68.437470,-38.471764


In [228]:
len(produccion_nonula['idpozo'].unique())

27715

In [231]:
produccion_nonula = produccion_nonula[produccion_nonula['idpozo'].isin(pozos['idpozo'])]
print(len(produccion_nonula['idpozo'].unique()))
produccion_nonula.to_csv("prod-no-nula.csv")

27695


In [232]:
produccion_nonula

,idempresa,anio,mes,idpozo,prod_pet,prod_gas,prod_agua,iny_agua,iny_gas,iny_co2,...,areayacimiento,cuenca,provincia,tipo_de_recurso,proyecto,clasificacion,subclasificacion,sub_tipo_recurso,fecha_data,index
250,YPF,2024,1,162223,3.080000,1.230000,2.460000,0.0,0.0,0.0,...,ESTANCIA CHOLITA,GOLFO SAN JORGE,Santa Cruz,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-01-31,250
251,YPF,2024,1,162222,169.320000,67.870000,135.190000,0.0,0.0,0.0,...,ESTANCIA CHOLITA,GOLFO SAN JORGE,Santa Cruz,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-01-31,251
252,YPF,2024,1,162221,135.460000,54.300000,108.160000,0.0,0.0,0.0,...,ESTANCIA CHOLITA,GOLFO SAN JORGE,Santa Cruz,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-01-31,252
253,YPF,2024,1,162220,64.400000,1.720000,132.790000,0.0,0.0,0.0,...,CAÑADON LEON,GOLFO SAN JORGE,Santa Cruz,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-01-31,253
254,YPF,2024,1,162188,257.570000,5.770000,629.950000,0.0,0.0,0.0,...,MANANTIALES BEHR,GOLFO SAN JORGE,Chubut,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-01-31,254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818723,ACO,2024,10,127447,78.133030,14.218152,1867.720154,0.0,0.0,0.0,...,25 DE MAYO - MEDANITO SUDESTE RN,NEUQUINA,Rio Negro,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-10-31,818723
818724,ACO,2024,10,71715,42.717569,12.885894,15.191638,0.0,0.0,0.0,...,25 DE MAYO - MEDANITO SUDESTE RN,NEUQUINA,Rio Negro,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-10-31,818724
818725,ACO,2024,10,127448,33.616528,13.036434,36.273066,0.0,0.0,0.0,...,25 DE MAYO - MEDANITO SUDESTE RN,NEUQUINA,Rio Negro,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-10-31,818725
818735,ACO,2024,10,134402,218.551046,12.750340,7162.803011,0.0,0.0,0.0,...,25 DE MAYO - MEDANITO SUDESTE RN,NEUQUINA,Rio Negro,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-10-31,818735
